## Imports

In [103]:
# Importing other packages
import timeit
import pandas as pd
import numpy as np
import neptune
import tempfile
import seaborn as sns
import matplotlib.pyplot as plt

import ipywidgets as widgets


import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "browser"

## Load data

In [104]:
df = pd.read_csv(f'./data/amazon_22.csv', parse_dates=['EffectiveDate'])

In [105]:
df['instanceFamily'].value_counts()

Memory optimized                   213171
Storage optimized                  131324
General purpose                    115748
Compute optimized                   84582
GPU instance                        24350
FPGA Instances                       1040
Machine Learning ASIC Instances       128
Name: instanceFamily, dtype: int64

In [106]:
df['TermType'].value_counts()

Reserved    570343
Name: TermType, dtype: int64

### Find unique instance types & count

In [107]:
instanceTypes = df['instanceType'].value_counts()

In [108]:
# exclude some instance families
df = df[(df['instanceFamily'] != "FPGA Instances") & (
    df['instanceFamily'] != "Machine Learning ASIC Instances")]

### Choose instanceType

In [109]:
# choose instance specific instance type

# df = df[(df['instanceType'] == "r5d.8xlarge")]

### Count unique Locations

In [110]:
df['Location'].value_counts()

AsiaPacific     178379
EU              137095
USEast           75870
USWest           59045
AWSGovCloud      33972
SouthAmerica     19694
Canada           19568
Europe           16656
MiddleEast       16352
Africa           12544
Name: Location, dtype: int64

In [111]:
df['Location'].value_counts()

AsiaPacific     178379
EU              137095
USEast           75870
USWest           59045
AWSGovCloud      33972
SouthAmerica     19694
Canada           19568
Europe           16656
MiddleEast       16352
Africa           12544
Name: Location, dtype: int64

In [112]:
# # keep only the countries that have data in all years
# df = df[(df['Location'] != "Africa") & (df['Location'] != "MiddleEast") & (df['Location']
#                                                                            != "Canada") & (df['Location'] != "SouthAmerica") & (df['Location'] != "AWSGovCloud")]

In [113]:
# Convert EffectiveDate column to datetime format
df['EffectiveDate'] = pd.to_datetime(df['EffectiveDate'])


In [114]:
df = df[df['year'] != 2023]

In [115]:
df

,SKU,OfferTermCode,RateCode,PricePerUnit,instanceType,instanceFamily,LeaseContractLength,PurchaseOption,OfferingClass,Product Family,...,Memory,Tenancy,operatingSystem,License Model,year,TermType,Network Performance,EffectiveDate,DiskType,StorageSize
0,222AY99RA8W7WFR4,38NPMPTW36,222AY99RA8W7WFR4.38NPMPTW36.6YS6EN2CT7,0.0605,r6gd.medium,Memory optimized,3,Partial Upfront,standard,Compute Instance,...,8,Dedicated,Red Hat Enterprise Linux with HA,No License required,2021,Reserved,10.0,2021-03-01,NVMe SSD,59
1,222AY99RA8W7WFR4,4NA7Y494T4,222AY99RA8W7WFR4.4NA7Y494T4.6YS6EN2CT7,0.1403,r6gd.medium,Memory optimized,1,No Upfront,standard,Compute Instance,...,8,Dedicated,Red Hat Enterprise Linux with HA,No License required,2021,Reserved,10.0,2021-03-01,NVMe SSD,59
2,222AY99RA8W7WFR4,7NE97W5U4E,222AY99RA8W7WFR4.7NE97W5U4E.6YS6EN2CT7,0.1466,r6gd.medium,Memory optimized,1,No Upfront,convertible,Compute Instance,...,8,Dedicated,Red Hat Enterprise Linux with HA,No License required,2021,Reserved,10.0,2021-03-01,NVMe SSD,59
3,222AY99RA8W7WFR4,BPH4J8HBKS,222AY99RA8W7WFR4.BPH4J8HBKS.6YS6EN2CT7,0.1231,r6gd.medium,Memory optimized,3,No Upfront,standard,Compute Instance,...,8,Dedicated,Red Hat Enterprise Linux with HA,No License required,2021,Reserved,10.0,2021-03-01,NVMe SSD,59
4,222AY99RA8W7WFR4,CUZHX8X6JH,222AY99RA8W7WFR4.CUZHX8X6JH.6YS6EN2CT7,0.0721,r6gd.medium,Memory optimized,1,Partial Upfront,convertible,Compute Instance,...,8,Dedicated,Red Hat Enterprise Linux with HA,No License required,2021,Reserved,10.0,2021-03-01,NVMe SSD,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570338,ZZZU9HQBJNWNW7AD,BPH4J8HBKS,ZZZU9HQBJNWNW7AD.BPH4J8HBKS.6YS6EN2CT7,0.1470,c5ad.xlarge,Compute optimized,3,No Upfront,standard,Compute Instance,...,8,Shared,RHEL,No License required,2020,Reserved,10.0,2020-06-01,NVMe SSD,150
570339,ZZZU9HQBJNWNW7AD,CUZHX8X6JH,ZZZU9HQBJNWNW7AD.CUZHX8X6JH.6YS6EN2CT7,0.1340,c5ad.xlarge,Compute optimized,1,Partial Upfront,convertible,Compute Instance,...,8,Shared,RHEL,No License required,2020,Reserved,10.0,2020-06-01,NVMe SSD,150
570340,ZZZU9HQBJNWNW7AD,HU7G6KETJZ,ZZZU9HQBJNWNW7AD.HU7G6KETJZ.6YS6EN2CT7,0.1210,c5ad.xlarge,Compute optimized,1,Partial Upfront,standard,Compute Instance,...,8,Shared,RHEL,No License required,2020,Reserved,10.0,2020-06-01,NVMe SSD,150
570341,ZZZU9HQBJNWNW7AD,R5XV2EPZQZ,ZZZU9HQBJNWNW7AD.R5XV2EPZQZ.6YS6EN2CT7,0.1080,c5ad.xlarge,Compute optimized,3,Partial Upfront,convertible,Compute Instance,...,8,Shared,RHEL,No License required,2020,Reserved,10.0,2020-06-01,NVMe SSD,150


In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 554079 entries, 0 to 570342
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   SKU                  554079 non-null  object        
 1   OfferTermCode        554079 non-null  object        
 2   RateCode             554079 non-null  object        
 3   PricePerUnit         554079 non-null  float64       
 4   instanceType         554079 non-null  object        
 5   instanceFamily       554079 non-null  object        
 6   LeaseContractLength  554079 non-null  int64         
 7   PurchaseOption       554079 non-null  object        
 8   OfferingClass        554079 non-null  object        
 9   Product Family       554079 non-null  object        
 10  Location             554079 non-null  object        
 11  Current Generation   554079 non-null  object        
 12  vCPU                 554079 non-null  int64         
 13  Memory        

In [117]:
# Extract the year and quarter from the Date column
df['year'] = df['EffectiveDate'].dt.year
df['Quarter'] = df['EffectiveDate'].dt.quarter

# Combine year and quarter into a single column
df['YearQuarter'] = df['year'].astype(
    str) + '-Q' + df['Quarter'].astype(str)

In [118]:
# Sort the DataFrame by 'YearQuarter' column in ascending order
df = df.sort_values('YearQuarter')

# Reset the index
df = df.reset_index(drop=True)

# Convert 'YearQuarter' column to PeriodIndex with quarterly frequency
df['YearQuarter'] = pd.PeriodIndex(df['YearQuarter'], freq='Q')

# Convert Period values to strings
df['YearQuarter'] = df['YearQuarter'].astype(str)

## Export dataframe with quarters

In [88]:
# Group RHEL & SUSE to Linux and rename Red Hat Enterprise Linux with HA
df = df.replace({'RHEL': 'Linux', 'SUSE': 'Linux', "Red Hat Enterprise Linux with HA": "RHEL_HA"})

# replace USEast and USWest to US and EU to Europe
df = df.replace({'USEast': 'US', 'USWest': 'US', 'EU': 'Europe'})

df.rename(columns = {'Network Performance': 'networkPerformance'},inplace=True)

df.to_csv(f'./data/amazon_22_quarters.csv', index=False)

## Calculate statistical metrics 

In [17]:
# Calculate statistic metricsby group
stats = df.groupby(['YearQuarter', 'Location', 'instanceFamily'])['PricePerUnit'].agg(
    ['mean', 'std', 'median', 'max', 'min']).reset_index()
stats.rename(columns={'mean': 'mean_price', 'std': 'std_deviation',
             'max': 'maximum', 'min': 'minimum'}, inplace=True)


# # Calculate statistic metricsby group
# stats = df.groupby(['year', 'instanceFamily'])['PricePerUnit'].agg(
#     ['mean', 'std', 'median', 'max', 'min']).reset_index()
# stats.rename(columns={'mean': 'mean_price', 'std': 'std_deviation',
#              'max': 'maximum', 'min': 'minimum'}, inplace=True)

In [18]:
stats['YearQuarter'].value_counts()

2021-Q1    39
2020-Q2    27
2016-Q4    24
2017-Q4    24
2019-Q3    22
2018-Q3    21
2019-Q2    21
2018-Q1    20
2016-Q3    18
2022-Q2    18
2021-Q2    18
2018-Q4    17
2022-Q3    16
2022-Q4    16
2020-Q3    15
2021-Q3    12
2022-Q1    12
2017-Q2    11
2019-Q4    10
2020-Q4     9
2018-Q2     9
2021-Q4     9
2020-Q1     8
2019-Q1     7
2017-Q1     5
2017-Q3     1
Name: YearQuarter, dtype: int64

In [19]:
# create a list for inst families
instFamilies = df['instanceFamily'].unique()
instFamilies

array(['Memory optimized', 'Compute optimized', 'Storage optimized',
       'GPU instance', 'General purpose'], dtype=object)

In [20]:

# # Loop through each instanceFamily and save the corresponding subset as a new DataFrame
# for instance_family in instFamilies:
#     subset = df[df['instanceFamily'] == instance_family]
#     subset.to_csv(f'./data/{instance_family}_subset.csv', index=False)

# Plots

## Plot specific instance type (by user)

In [23]:
import plotly.express as px

# Take instanceType as input from the user
instance_type = input("Enter the instance type: ")

# Filter the data for the specified instance type
filtered_data = df[df['instanceType'] == instance_type]

# # Extract the year and quarter from the Date column
# filtered_data['Year'] = filtered_data['EffectiveDate'].dt.year
# filtered_data['Quarter'] = filtered_data['EffectiveDate'].dt.quarter

# # Combine year and quarter into a single column
# filtered_data['YearQuarter'] = filtered_data['Year'].astype(
#     str) + '-Q' + filtered_data['Quarter'].astype(str)

# Calculate statistical metrics by year and quarter
stats_filtered = filtered_data.groupby('YearQuarter')['PricePerUnit'].agg(
    ['mean', 'std', 'median', 'max', 'min']).reset_index()
stats_filtered.rename(columns={'mean': 'mean_price', 'std': 'std_deviation',
                               'max': 'maximum', 'min': 'minimum'}, inplace=True)

# Create the line plot
fig = px.line(stats_filtered, x='YearQuarter', y='mean_price',
              template='presentation',
              title=f'Evolution of median price for {instance_type}')

fig.update_traces(mode='lines+markers')

fig.update_layout(
    xaxis=dict(title='Year and Quarter',
               tickfont=dict(size=12, family='Arial')),
    yaxis=dict(title='Price per Unit', tickfont=dict(size=12, family='Arial')),
    legend=dict(orientation='h', yanchor='bottom',
                y=1.02, xanchor='right', x=1),
    title=dict(font=dict(size=14, family='Arial')),
    font=dict(size=12, family='Arial')
)

fig.show()

## Evolution of median for specific instance - all Locations (by user)

In [28]:
import pandas as pd
import plotly.express as px

instance_family = input("Enter the instance family: ")

filtered_data = df[df['instanceFamily'] == instance_family]

# Convert 'YearQuarter' column to PeriodIndex with quarterly frequency
filtered_data['YearQuarter'] = pd.PeriodIndex(filtered_data['YearQuarter'], freq='Q')

# Convert Period values to strings
filtered_data['YearQuarter'] = filtered_data['YearQuarter'].astype(str)

# Calculate statistical metrics by year and quarter
stats_filtered = filtered_data.groupby('YearQuarter')['PricePerUnit'].agg(
    ['mean', 'std', 'median', 'max', 'min']).reset_index()
stats_filtered.rename(columns={'mean': 'mean_price', 'std': 'std_deviation',
                               'max': 'maximum', 'min': 'minimum'}, inplace=True)

fig = px.line(stats_filtered, x='YearQuarter', y='median',
              template='presentation',
              title=f'Evolution of median price for {instance_family} Instance Family')

fig.update_traces(mode='lines+markers')

fig.update_layout(
    xaxis=dict(title='Year and Quarter',
               tickfont=dict(size=20, family='Arial')),
    yaxis=dict(title='Price per Unit', tickfont=dict(size=20, family='Arial')),
    legend=dict(orientation='h', yanchor='bottom',
                y=1.02, xanchor='right', x=1),
    title=dict(font=dict(size=24, family='Arial')),
    font=dict(size=18, family='Arial')
)

fig.show()


/tmp/ipykernel_414261/1224630238.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_414261/1224630238.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Plot specific instance Family per Location

In [37]:
import plotly.express as px

# Calculate statistic metrics by group
stats = df.groupby(['YearQuarter', 'Location', 'instanceFamily'])['PricePerUnit'].agg(
    ['mean', 'std', 'median', 'max', 'min']).reset_index()
stats.rename(columns={'mean': 'mean_price', 'std': 'std_deviation',
                      'max': 'maximum', 'min': 'minimum'}, inplace=True)

# Convert 'YearQuarter' column to PeriodIndex with quarterly frequency
stats['YearQuarter'] = pd.PeriodIndex(stats['YearQuarter'], freq='Q')

# Convert Period values to strings
stats['YearQuarter'] = stats['YearQuarter'].astype(str)

compute_optimized_data = stats[stats['instanceFamily'] == 'Compute optimized']

# Fill missing data points with NaN values
locations = compute_optimized_data['Location'].unique()
quarters = compute_optimized_data['YearQuarter'].unique()
combined_index = pd.MultiIndex.from_product([locations, quarters], names=['Location', 'YearQuarter'])
compute_optimized_data = compute_optimized_data.set_index(['Location', 'YearQuarter']).reindex(combined_index).reset_index()

# Sort the DataFrame by 'YearQuarter' column
compute_optimized_data.sort_values(by='YearQuarter', inplace=True)

# Extract 'median' column as a Series
median_series = compute_optimized_data['median']

# Interpolate missing values in the Series
interpolated_median = median_series.interpolate()

# Update the original DataFrame with the interpolated values
compute_optimized_data['median'] = interpolated_median

# Filter out interpolated values before the first valid data point for each location
compute_optimized_data = compute_optimized_data.groupby('Location').apply(lambda x: x.loc[x['median'].first_valid_index():]).reset_index(drop=True)

# Create the line plot
fig = px.line(compute_optimized_data, x='YearQuarter', y='median', color='Location',
              template='presentation',
              title='Evolution of median price for Compute Optimized Instance Family')

fig.update_traces(mode='lines+markers')

fig.update_layout(
    xaxis=dict(title='Year', tickfont=dict(size=20, family='Arial')),
    yaxis=dict(title='Price per Unit', tickfont=dict(size=20, family='Arial')),
    legend=dict(orientation='h', yanchor='bottom',
                y=1.02, xanchor='right', x=1),
    title=dict(font=dict(size=24, family='Arial')),
    font=dict(size=18, family='Arial')
)

fig.show()


/tmp/ipykernel_414261/3630575260.py:36: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)



In [ ]:
import plotly.express as px

compute_optimized_data = stats[stats['instanceFamily'] == 'Compute optimized']

fig = px.line(compute_optimized_data, x='YearQuarter', y=['median'],
              template='presentation',
              title='Evolution of median price for Compute Optimized Instance Family',
              hover_name='instanceFamily')

fig.update_traces(mode='lines+markers')

fig.update_layout(
    xaxis=dict(title='Year', tickfont=dict(size=20, family='Arial')),
    yaxis=dict(title='Price per Unit', tickfont=dict(size=20, family='Arial')),
    legend=dict(orientation='h', yanchor='bottom',
                y=1.02, xanchor='right', x=1),
    title=dict(font=dict(size=24, family='Arial')),
    font=dict(size=18, family='Arial')
)

fig.show()

In [ ]:
fig = px.scatter(df, x=df.index, y=df['PricePerUnit'], template='presentation',
                 labels=dict(x='Data points', y='Price per unit'),
                 color='PricePerUnit', title='Density of Price per year', facet_col='year',
                 category_orders={'year': [2016, 2017, 2018, 2019, 2020, 2021, 2022]})
# Modify font properties
fig.update_layout(
    font=dict(
        family='Arial',  # Change to your desired font family
        size=16,
    )
)  # Change to your desired font size
# fig.write_html("results/stats/density.html")
fig.show()

In [ ]:
import plotly.express as px

fig = px.scatter_3d(df, x='year', y='instanceFamily', z='PricePerUnit', template='presentation',
                    labels=dict(z='Price per Hour'),
                    color='instanceFamily', title='Density of Price per Year',
                    category_orders={'year': [2016, 2017, 2018, 2019, 2020, 2021, 2022]})

fig.update_layout(
    legend=dict(orientation='h', yanchor='bottom',
                y=-0.1, xanchor='left', x=0),
    font=dict(size=16, family='Arial'),
    scene=dict(
        xaxis=dict(title_font=dict(size=20, family='Arial')),
        yaxis=dict(title_font=dict(size=20, family='Arial')),
        zaxis=dict(title_font=dict(size=20, family='Arial')),
        xaxis_title='', yaxis_title='', zaxis_title='Price($) per Hour'
    ),
    title=dict(font=dict(size=24, family='Arial'))
)

fig.update_traces(marker=dict(size=8))

fig.show()

In [ ]:
import plotly.express as px

fig = px.scatter(df, x='year', y='instanceFamily', color='PricePerUnit', template='presentation',
                 labels=dict(x='Year', y='instanceFamily',
                             color='Price per Hour'),
                 color_continuous_scale='Viridis', title='Density of Price per Year')

fig.update_layout(
    legend=dict(orientation='v', yanchor='bottom',
                y=0.5, xanchor='right', x=0),
    font=dict(size=16, family='Arial'),
    xaxis=dict(title_font=dict(size=20, family='Arial')),
    yaxis=dict(title_font=dict(size=20, family='Arial')),
    title=dict(font=dict(size=24, family='Arial'))
)

fig.update_traces(marker=dict(size=8))

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Histogram2d(
    x=df['year'],
    y=df['instanceFamily'],
    colorscale='YlGnBu',
    zauto=True,
    nbinsx=len(df['year'].unique()),
    nbinsy=len(df['instanceFamily'].unique()),
))

fig.update_layout(
    title='Density of Offers per Year',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Instance Family'),
    coloraxis_colorbar=dict(title='Density'),
)

fig.show()


# 'Viridis'
# 'Cividis'
# 'Hot'
# 'Jet'
# 'Rainbow'
# 'Electric'
# 'Blues'
# 'Greens'
# 'Reds'
# 'YlOrRd'
# 'YlGnBu'